In [54]:
from skl2onnx import to_onnx
from sklearn.ensemble import RandomForestClassifier
import sqlalchemy as db
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from skl2onnx.common.data_types import FloatTensorType, Int16TensorType, DoubleTensorType
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report  

from mclLib.featurenames import *
from mclLib.modelpostfix import *
from mclLib.cleaner import *
from mclLib.server_info import *
from mclLib.tester import *

In [2]:

engine = create_engine(mysql_server_uri)
conn = engine.connect()


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [107]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & (br_full_data['dTradeTime'] >= '2023-02-24')
br = br_full_data[get_filter]

''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

model_name = 'RF_max30_006_v3'

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y_condition = ((br['fMaxPowerAfterBuyWhile30'] <= 0.03))# | ((br['lTotalBuyEndPrice'] - br['lTotalBuyPrice']) <= 1000000000))
y = np.where(y_condition, 1, 0)

In [108]:
from collections import Counter
Counter(y)

Counter({1: 76553, 0: 31258})

In [51]:
# smote = SMOTE(sampling_strategy=0.85)
# X, y = smote.fit_resample(X, y)

In [109]:
rf = RandomForestClassifier(n_estimators=250, max_depth=5)
rf.fit(X, y)

RandomForestClassifier(max_depth=5, n_estimators=250)

In [110]:
onx = to_onnx(model=rf, options={'zipmap': False},
              initial_types=[('input', DoubleTensorType([None, 102]))],
              final_types=[('output', DoubleTensorType([None])),
                           ('prob', DoubleTensorType([None, 2]))],
              target_opset=17)

with open(onnx_path + model_name + onnx_, "wb") as f:
    f.write(onx.SerializeToString())

In [111]:
cleanAll()

In [104]:
y_pred = rf.predict(X)

In [105]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.36      0.47     21080
           1       0.67      0.87      0.76     31711

    accuracy                           0.67     52791
   macro avg       0.67      0.62      0.61     52791
weighted avg       0.67      0.67      0.64     52791



In [106]:
testClassification(y, [y_pred])

총량 :  52791
0 :  21080 , 비율 :  39.931048853024194 (%)
1 :  31711 , 비율 :  60.06895114697581 (%)

============ predict 0 =============
총 횟수 :  11619 ,  타겟기준 :  0.0
실제 0 :  7653
실제 1 :  3966
정답비율 :  65.86625355021947 (%)

============ predict 1 =============
총 횟수 :  41172 , 타겟기준 :  1.0
실제 1 :  27745
실제 0 :  13427
정답비율 :  67.38803070047605 (%)

